In [1]:
import spacy
from spacy import displacy

In [2]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [3]:
import json
 
with open('data/flight_spacy_dataset.json', 'r') as f:
    training_data = json.load(f)

In [4]:
training_data[0]

{'text': 'Show me flights from Barcelona to Dubai on Apr 02.',
 'entities': [[21, 30, 'SOURCE'],
  [34, 39, 'DESTINATION'],
  [43, 49, 'DEPART_DATE']]}

In [5]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en")
doc_bin = DocBin()

In [6]:
from spacy.util import filter_spans

for training_example  in tqdm(training_data[0:4000]): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("train.spacy") 

100%|██████████| 4000/4000 [00:01<00:00, 2785.13it/s]


In [7]:
from spacy.util import filter_spans

for training_example  in tqdm(training_data[4001:5000]): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("test.spacy") 

100%|██████████| 999/999 [00:00<00:00, 2919.31it/s]


In [9]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only), install the
spacy-transformers package and re-run this command. The config generated now
does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [11]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./test.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     64.22    0.00    0.00    0.00    0.00
  0     200        213.49   2829.59   99.32   99.43   99.21    0.99
  0     400         17.06     17.92  100.00  100.00  100.00    1.00
  1     600          1.40      1.60  100.00  100.00  100.00    1.00
  1     800         10.28      1.48  100.00  100.00  100.00    1.00
  2    1000          7.41      3.61  100.00  100.00  100.00    1.00
  3    1200          0.00      0.00  100.00  100.00  100.00    1.00
  3    1400         10.61      3.37  100.00  100.00  100.00    1.00
  5    1600          7.14      1.18 

In [12]:

nlp = spacy.load("model-best")
doc = nlp("I want a round-trip flight from Los Angeles to Bangalore on 2025-11-29.")

print(doc.ents)
displacy.render(doc, style="ent", jupyter=True)

(round-trip flight, Los Angeles, Bangalore, 2025-11-29)
